In [1]:
import numpy as np
import opt_einsum as oe
from opt_einsum import contract

In [2]:
batch = 10
SRC_NT = 10
TGT_NT = 10
NT = SRC_NT * TGT_NT
r = 2
n = 10
w = 10

# H = np.random.randn(batch, NT, r)  # , A[i] -> R
H = np.random.randn(batch, TGT_NT, SRC_NT, r)  # , A[i] -> R
TL_NT = np.random.randn(batch, r, SRC_NT, TGT_NT) # R, j -> B
TR_NT = np.random.randn(batch, r, SRC_NT, TGT_NT) # R, k -> C
SL = np.random.randn(batch, r, SRC_NT, SRC_NT) # R, i -> j
SR = np.random.randn(batch, r, SRC_NT, SRC_NT) # R, i -> k

Y = np.random.randn(batch, n, w, TGT_NT, SRC_NT)
Z = np.random.randn(batch, n, w, TGT_NT, SRC_NT)

In [3]:
oe.contract_path('qair,qrjb,qrkc,qrij,qrik,qnwbj,qnwck->qnai', 
    H, TL_NT, TR_NT, SL, SR, Y, Z, optimize='optimal') 


# SRC_NT and n matters

([(1, 5), (1, 4), (1, 2), (1, 2), (1, 2), (0, 1)],
   Complete contraction:  qair,qrjb,qrkc,qrij,qrik,qnwbj,qnwck->qnai
          Naive scaling:  10
      Optimized scaling:  6
       Naive FLOP count:  1.400e+10
   Optimized FLOP count:  1.660e+6
    Theoretical speedup:  8.434e+3
   Largest intermediate:  2.000e+4 elements
 --------------------------------------------------------------------------------
 scaling        BLAS                current                             remaining
 --------------------------------------------------------------------------------
    6              0      qnwbj,qrjb->qnwjr    qair,qrkc,qrij,qrik,qnwck,qnwjr->qnai
    6              0      qnwck,qrkc->qnwkr      qair,qrij,qrik,qnwjr,qnwkr->qnai
    5              0       qrik,qrij->qrikj          qair,qnwjr,qnwkr,qrikj->qnai
    6              0     qnwkr,qnwjr->qnkrj                qair,qrikj,qnkrj->qnai
    6              0      qnkrj,qrikj->qnri                       qair,qnri->qnai
    5         

In [4]:
H = np.random.randn(batch, NT, r)
L = np.random.randn(batch, NT, r)
R = np.random.randn(batch, NT, r)
Y = np.random.randn(batch, n, w, NT)
Z = np.random.randn(batch, n, w, NT)

oe.contract_path('qar,qbr,qcr,qnwb,qnwc->qna', 
    H, L, R, Y, Z, optimize='optimal') 

([(1, 3), (1, 2), (1, 2), (0, 1)],
   Complete contraction:  qar,qbr,qcr,qnwb,qnwc->qna
          Naive scaling:  7
      Optimized scaling:  5
       Naive FLOP count:  1.000e+10
   Optimized FLOP count:  8.440e+5
    Theoretical speedup:  1.185e+4
   Largest intermediate:  1.000e+4 elements
 --------------------------------------------------------------------------------
 scaling        BLAS                current                             remaining
 --------------------------------------------------------------------------------
    5              0         qnwb,qbr->qnwr                qar,qcr,qnwc,qnwr->qna
    5              0         qnwc,qcr->qnwr                    qar,qnwr,qnwr->qna
    4              0         qnwr,qnwr->qnr                          qar,qnr->qna
    4              0           qnr,qar->qna                              qna->qna)

In [72]:
# 